In [1]:
import bson 
import json
import numpy as np
import pandas as pd
import time
import sys
import scipy.sparse
import pickle
import sys
import re

In [2]:
from tqdm import tqdm_notebook as tqdm
import time
from mtranslate import translate
from langdetect import detect
from langdetect import detect_langs
from numba import njit
from functools import wraps
import cProfile

In [3]:
with open("linkedin_db/profile.bson",'rb') as f: 
    data_profile = bson.decode_all(f.read()) 
    f.close()

In [4]:
with open("linkedin_db/skill.bson",'rb') as f: 
    data_skill = bson.decode_all(f.read()) 
    f.close()

In [4]:
f2 = open("all_top_skills_final_fre.txt","rb")
fre_save = pickle.load(f2)
f2.close()
f2 = open("after_delete_keysave.txt","rb")
key_save = pickle.load(f2)
f2.close()
f= file('word2vec_model_allskills', 'rb')
word2vec=pickle.load(f)

D:\anaconda2\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [4]:
f2 = open("all_top_skills_final_key_backup.txt","rb")
fre_save = pickle.load(f2)
f2.close()
f2 = open("all_top_skills_final_fre_backup.txt","rb")
key_save = pickle.load(f2)
f2.close()
f= file('word2vec_model_allskills', 'rb')
word2vec=pickle.load(f)

D:\anaconda2\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [37]:
f2 = open("similiarty.txt","rb")
uni1 = pickle.load(f2)
f2.close()
f2 = open("less_similiarty.txt","rb")
uni2 = pickle.load(f2)
f2.close()
f2 = open("simi_order.txt","rb")
simi_ana = pickle.load(f2)
f2.close()

In [8]:
print len(fre_save)
print len(key_save)

6855
6806


In [71]:
def all_lower(L1):
    return [s.lower() for s in L1]

def all_upper(L1):
    return [s.upper() for s in L1]


A Jupyter Widget

In [73]:
########################################################################
# data reconstruction(top skills):merge,translate,delete and so on
##########################################################################

Key_save,Fre_save = {},{}
start = time.clock()
#for i in tqdm(range(len(data_profile))):
for i in tqdm(range(0,100)):
    if 'skills' in (data_profile[i].keys()):
        for skills_divided in (data_profile[i]['skills']):                        
            #for title in skill_titlt:
                  #if skills_divided['title'] in skill_titlt:
            for j in (range(len(skills_divided['skills']))):
                #print skills_divided['skills'][j]['title']
                #print '\n'
                ty_lang = detect(skills_divided['skills'][j]['title']).encode('unicode-escape').decode('string_escape')
                ################################ so so so inaccurate for detect
                if ty_lang != 'en':
                    to_translate = skills_divided['skills'][j]['title'].encode('utf-8')
                    #result = translate(to_translate,'en',ty_lang).lower().replace(' ','_')
                    result = translate(to_translate).lower().replace(' ','_')
                    ###################### no change for wrong spelling
                    if '\\' in repr(result):
                        result = translate(to_translate,'en',ty_lang).lower().replace(' ','_')

                else:
                    result = skills_divided['skills'][j]['title'].lower().replace(' ','_')    
                if result in (Key_save.keys()):
                    if skills_divided['skills'][j]['title'] not in Key_save[result]:
                        Key_save[result].append(skills_divided['skills'][j]['title']) 
                    Fre_save[result] += 1
                    '''
                    if title == u'Top Skills':
                        Fre_save[result] += 1 
                    else:
                        Fre_save[result] += 0.5
                    '''
                else:
                    Key_save[result] = [skills_divided['skills'][j]['title']]
                    Fre_save[result] =1  

''' 
if skills_divided['title'] ==  u'Connaissances du secteur' :
    for j in (range(len(skills_divided['skills']))):
        #print skills_divided['skills'][j]['title']
        #print '\n'
        ty_lang = detect(skills_divided['skills'][j]['title']).encode('unicode-escape').decode('string_escape')
        ################################ so so so inaccurate for detect
        if ty_lang != 'en':
            to_translate = skills_divided['skills'][j]['title'].encode('utf-8')
            #result = translate(to_translate,'en',ty_lang).lower().replace(' ','_')
            result = translate(to_translate).lower().replace(' ','_')
            ###################### no change for wrong spelling
            if '\\' in result:
                result = translate(to_translate,'en',ty_lang).lower().replace(' ','_')

        else:
            result = skills_divided['skills'][j]['title'].lower().replace(' ','_')    
        if result in (key_save.keys()):
            if skills_divided['skills'][j]['title'] not in key_save[result]:
                key_save[result].append(skills_divided['skills'][j]['title'])
            fre_save[result] +=1                   
        else:
            key_save[result] = [skills_divided['skills'][j]['title']]
            fre_save[result] =1 
'''  

print(time.clock()-start)

A Jupyter Widget


269.745390933


In [7]:
def levenshtein1(source, target):
    if len(source) < len(target):
        return levenshtein1(target, source)
 
    # So now we have len(source) >= len(target).
    if len(target) == 0:
        return len(source)
    source = numpy.array(tuple(source))
    target = numpy.array(tuple(target))
 
    # We use a dynamic programming algorithm, but with the
    # added optimization that we only need the last two rows
    # of the matrix.
    previous_row = numpy.arange(target.size + 1)
    for s in source:
        # Insertion (target grows longer than source):
        current_row = previous_row + 1
 
        # Substitution or matching:
        # Target and source items are aligned, and either
        # are different (cost of 1), or are the same (cost of 0).
        current_row[1:] = numpy.minimum(
                current_row[1:],
                numpy.add(previous_row[:-1], target != s))
 
        # Deletion (target grows shorter than source):
        current_row[1:] = numpy.minimum(
                current_row[1:],
                current_row[0:-1] + 1)
        previous_row = current_row
    return previous_row[-1]
 
def levenshtein2(s1, s2):
    if len(s1) < len(s2):
        return levenshtein2(s2, s1)
 
    # len(s1) >= len(s2)
    if len(s2) == 0:
        return len(s1)
 
    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1 # j+1 instead of j since previous_row and current_row are one character longer
            deletions = current_row[j] + 1       # than s2
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
 
    return previous_row[-1]
 
def levenshtein3(s, t):
        if s == t: return 0
        elif len(s) == 0: return len(t)
        elif len(t) == 0: return len(s)
        v0 = [None] * (len(t) + 1)
        v1 = [None] * (len(t) + 1)
        for i in range(len(v0)):
            v0[i] = i
        for i in range(len(s)):
            v1[0] = i + 1
            for j in range(len(t)):
                cost = 0 if s[i] == t[j] else 1
                v1[j + 1] = min(v1[j] + 1, v0[j + 1] + 1, v0[j] + cost)
            for j in range(len(v0)):
                v0[j] = v1[j]
 
        return v1[len(t)]
 



In [85]:
############################################
# to judge the edit distance between 2 words
############################################
uni1,uni2,uni3 = [],[],[]
simi_ana = []
order = 0
#m,n,save,order= 0,0,0,0
#save_value = np.zeros(len(test)*len(test)).reshape(len(test),len(test))
for key1,value1 in tqdm(key_sort[:3000]):
    for key2,value2 in key_sort[order+1:3000]:
        '''
        group_1 = key1.split()
        group_2 = key2.split()
        len1 = len(group_1)
        len2 = len(group_2)
        for i in range(len1):
            ratio = 200
            for j in range(len2):
                save = ratio
                ratio = levenshtein3(group_1[i],group_2[j])   ##/(np.mean([len(group_1[i]),len(group_2[j])]))
                minim = min(save,ratio)
            mini.append(minim) 
        result = sum(mini)/(np.mean([len(key1),len(key2)]))
        mini=[]
        '''
        result = levenshtein3(key1,key2)/(np.mean([len(key1),len(key2)]))
        #m += 1
    #n += 1 
    #save_value[n][m] = result
        if result <0.1:
            #sum_value = test[key1] + test[key2]
            #select[unit] = sum_value
            uni1.append( ' '.join([key1,key2]))
        if 0.1 <= result <0.2:
            #sum_value = test[key1] + test[key2]
            #select[unit] = sum_value
            uni2.append( ' '.join([key1,key2]))
        if 0.2 <=result <0.3:
            #sum_value = test[key1] + test[key2]
            #select[unit] = sum_value
            uni3.append( ' '.join([key1,key2]))
        
        if key1 in key2 or key2 in key1:
            if np.mean([len(key1),len(key2)])>=9.5:
                '''
                aa=[a[i] for i in range(len(key1))]
                bb=[b[i] for i in range(len(key2))]
                diff = max(len(aa),len(bb)) -  min(len(aa),len(bb))
                (longer, shorter) = (aa,bb) if len(aa) >= len(bb) else (bb,aa)
                if len(longer-shorter) == diff:
                    simi_ana.append(' '.join([key1,key2]))
                '''
                if np.abs(len(key1) - len(key2))<= 3:
                    simi_ana.append(' '.join([key1,key2]))
                
    order+=1

'''      
ratio = levenshtein_distance(key1,key2)/(np.mean([len(key1),len(key2)]))
a = wn.synset('Defense.n.01')
b = wn.synset('security.n.01')
score = a.path_similarity(b)
'''        

A Jupyter Widget

In [ ]:
###################################################################################
# choose out the skills which are almost same ant merge them by distance functioin
#################################################################################

for x in tqdm(skill_choosen):
    [key1,key2] = x.split()
    if key1 in key_save.keys() and key2 in key_save.keys():
    # in case we have alreday deleted the key eralier
        if fre_save[key1] >= fre_save[key2]:
            key_save[key1] += key_save[key2]
            fre_save[key1] += fre_save[key2]
            del key_save[key2]
            del fre_save[key2]
        else:
            key_save[key2] += key_save[key1]
            fre_save[key2] += fre_save[key1]
            del key_save[key1]
            del fre_save[key1]
    else:
        print (key1,key2)
    

In [46]:
skill_choosen=[u'recruitment recruitments',
 u'business_development business_developement',
 u'machine_learning machine-learning',
 u'financial_analysis financial_analyses',
 u'negotiation negociation',
 #u'pack_office back_office',
 u'software_development software_developpment',
 u'data_mining datamining',
 u'architecture architectures',
 u'systems_engineering system_engineering',
 u'embedded_systems embedded_system',
 u'financial_modeling financial_modelling',
 u'operations_management operational_management',
 u'mathematics mathematica',
 u'internal_controls internal_control',
 u'internal_communications internal_communication',
 u'asset_management asset_managment',
 u'visual_basic_pour_applications_(vba) visual_basic_for_applications_(vba)',
 u'financial_audits financial_audit',
 u'capital_markets capital_market',
 u'crisis_communication crisis_communications',
 u'international_sales international_sale',
 u'business_objects business_object',
 u'business_objects businessobjects',
 u'virtualisation virtualization',
 u'materials_science material_science',
 u'political_sciences political_science',
 u'restaurant restaurants',
 u'optimization optimisation',
 u'information_system information_systems',
 u'web_marketing webmarketing',
 u'semiconductors semiconductor',
 u'cyber-security cybersecurity',
 u'life_sciences lifesciences',
 u'data_visualization data_visualisation',
 u'human_resources_development human_resource_development',
 u'anti_money_laundering anti-money_laundering',
 u'medical_device medical_devices',
 u'system_administrator system_administration',
 u'problem_solving problems_solving',
 u'aerodynamics aerodynamic',
 u'railway_signalling railway_signaling',
 u'electricity_markets electricity_market',
 u'commercial commercials',
 u'fundraising fund_raising',
 u'system_architecture systems_architecture',
 u'head_hunter head-hunter',
 u'user-centered_design user_centered_design',
 u'press_releases press_release',
 u'payment_systems payment_system',
 u'security_audits security_audit',
 u'thermodynamics thermodynamic',
 #u'btob_marketing btoc_marketing', @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
 u'human_resource_management human_resources_management',
 u'ohsas_18001 ohsas18001',
 u'functional_programming functional-programming',
 u'informatica informatics',
 u'scikit-learn scikit_learn',
 u'design_pattern design_patterns',
 u'call_centers call_center',
 u'neuroscience neurosciences',
 u'import-export import_export',
 u'checkpoint check_point',
 u'government governments',
 u'photovoltaic photovoltaics',
 #u'r_programming c_programming',
 u'linguistic linguistics',
 #u'external_communication internal_communication',
 u'railway_operations railway_operation',
              
              
              
              
              
              
 #u'project_management process_management',
 #u'project_management hr_project_management',####################################  merge？？？？？？？？？？？？？？？？？？
 #u'project_management property_management',
 #u'team_management lean_management',
 #u'team_management test_management',
 #u'python cython',
 u'finance finances',
 u'social_networks social_networking',
 u'teamwork team_work',
 u'marketing_communications marketing_et_communication',
 u'data_analysis data_analyst',
 #u'event_management talent_management',
 #u'event_management revenue_management',
 #u'event_management test_management',
 #u'event_management fleet_management',
 u'statistics statistical',
 #u'corporate_law corporate_tax',
 #u'talent_management sales_management',
 #u'talent_management test_management',
 #u'talent_management fleet_management',
 u'entrepreneurship intrapreneurship',
 u'achats achat',
 #u'it_strategy hr_strategy',
 u'renewable_energy renewable_energies',
 #u'contract_management content_management',
 #u'contract_management conflict_management',
 u'procurement e-procurement',
 #u'transportation transformation',
 #u'mechanical_engineering chemical_engineering',
 #u'international_relations international_taxation',
 #u'sales_management water_management',
 #u'sales_management skills_management',
 #u'sales_management stress_management',
 u'business_analysis business_analytics',
 u'business_analysis business_analyst',
 u'it_service_management service_management',
 u'it_service_management guest_service_management',
 u'start-ups startups',
 #u'project_engineering process_engineering',
 u'budgets budget',
 u'node.js nodejs',
 u'operations_management it_operations_management',
 u'litigation mitigation',       ########################## wrong spelling
 u'intellectual_property intellectual_property_law',
 #u'programming r_programming',
 #u'programming c_programming',
 u'web_design webdesign',
 #u'internal_audit external_audit',######################################################################
 #u'it_management hr_management',
 #u'it_management bid_management',
 u'sport sports',
 #u'cash_management waste_management',
 #u'programmation_web programmation_sas',
 #u'internal_communications external_communication',  ##############################################################
 #u'asset_management waste_management',
 #u'asset_management test_management',
 #u'asset_management fleet_management',
 #u'wealth_management waste_management',
 u'railway railways',
 #u'commercial_litigation commercial_animation',
 #u'innovation_management information_management',
 #u'international_business international_business_law', ################################################################
 #u'policy police',
 #u'lean_management brand_management',
 #u'lean_management fleet_management',
 #u'lean_management demand_management',
 u'photographie photography',
 u'defense defence',
 u'analysis analyses',
 #u'web_project_management hr_project_management',
 #u'web_project_management event_project_management',
 u'symfony symfony2',
 #u'international_law international_tax',
 #u'international_law international_sale',
 #'international_law international_m&a',
 #u'international_project_management operational_project_management',
 u'robotics cobotics',        ############################################# wrong spelling
 #u'prospection protection',
 #u'video_production radio_production',#######################################################
 u'business_process business_processes',
 #u'energy_management general_management',
 u'illustrator illustration',
 u'cross-functional_team_leadership multifunctional_team_leadership',
 #u'staff_management store_management',
 #u'staff_management turf_management',
 u'finance_management financial_management',
 #u'hotel_management water_management',
 #u'hotel_management soil_management',
 u'hotel hotels',
 u'real_estate_development real_estate_developer',
 u'supply_management supplies_management',
 u'j2ee_application_development web_application_development',  #@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
 u'j2ee_application_development agile_application_development',  #######################################################
 #u'business_law business_plan',
 u'business_process_improvement business_process_management',
 u'angular angular2',
 u'angular angular4',              
 u'artistic_direction artistic_director',
 #u'international_marketing operational_marketing', ##################################
 #u'commissioning decommissioning',
 u'customer_relations customer_relationship',
 u'design desig',              #################################################wrtong spelling
 u'analytics analytic',
 u'technical_recruiting technical_recruitment',
 u'product_manager project_manager',
 #u'procurement_management document_management',
 u'project_finance project_financing',
 u'cosmetics cosmetic',
 #u'ux_design ui_design', #######################################################
 #u'organic_chemistry inorganic_chemistry',
 u'maintenance_and_repair maintenance_&_repair',
 u'signaling signalling',
 u'medical_devices biomedical_devices',
 #u'sap_bw sap_bi',##################################
 u'operations_research operational_research',
 u'creativity creativit\xe9',
 u'iso_9001 iso_9000', ######################
 u'iso_27001 iso_27005',############################################
 #u'hr_transformation it_transformation',
 #u'hr_transformation transformation',
 #u'visual_communication digital_communication',
 u'java_se java_ee',       #################################################
 u'java_se javase',
 #u'c_language c++_language', ##########################
 u'health_services it_health_services',  ######################################
 u'commercial_banking commercial_bank',
 u'employer_brand employer_branding',
 #u'system_administration server_administration',
 #u'time_management store_management',
 #u'time_management sme_management',
 #u'international_trade international_tax',
 u'international_trade international_sale',
 u'purchase_management purchasing_management',
 u'facilities_management facility_management',
 u'hp_quality_center quality_center',    ########################################interesting          
 u'java_ee java_jee',  ###################################################
 #u'career_management water_management',
 #u'career_management order_management',
 u'textile textiles',
 #u'brand_management demand_management',
 u'windows_7 windows_8',
 u'fine_arts fine_art',
 #u'it_transformation transformation',
 #u'data_management water_management',
 #u'data_management waste_management',
 #u'water_management waste_management',
 #u'water_management order_management',
 #u'sports_management store_management',
 #u'financial_management it_financial_management',
 u'catia_v5 catia_v6',
 #u'transformation transformation_rh',
 u'human_relations human_relationship',
 u'angular_4 angular_2',
 u'angular_4 angular_5',
 #u'international_tax international_sale',##########################################
 #u'international_tax international_m&a',
 #u'waste_management test_management',
 u'symfony2 symfony_2',
 u'safety_management_systems security_management_systems',
 #u'drilling_engineering building_engineering',
 u'java_j2ee java_jee',
 #u'architectures architecture_si',
 #u'web_application_development agile_application_development',###############################################
 u'composite composites',
 u'key_performance_indicators performance_indicators',
 #u'skills_management soil_management',
 #u'store_management stress_management',
 #u'store_management turf_management',
 #u'store_management soil_management',
 u'pipeline pipelines',
 #u'requirements_management equipment_management',
 u'application_security web_application_security',
 u'application_security applicative_security',
 u'acoustics acoustic',
 u'programmation programmatic',
 #u'bid_management r&d_management',
 #u'sas_programming java_programming', ################################
 u'angular_2 angular_5',
 u'angular_2 angular2',
 #u'test_management stress_management',
 #u'test_management fleet_management',
 u'struts struts2',
 #u'business_creation business_relations',
 #u'training_management learning_management',
 u'handicap handicaps',
 #u'ios_development site_development',
 u'rail_operations railway_operations',
 u'predictive_analytics predictive_analysis',
 #u'architecture_si architecture_arm',
 #u'rail_transport air_transport', @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
 u'java/jee java_jee',
 #u'executive_management delegative_management',
 #u'sap_fi sap_bi', ##########################################
 #u'commercial_aviation commercial_animation',
 u'business_object businessobjects',
 u'english_bilingual english_(bilingual)',
 #u'applications_web applications_php',
 #u'game_development site_development',
 #u'game_development api_development',
 #u'radioprotection radio_production',
 #u'facility_management mobility_management', @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
 u'wireless_networking wireless_network',
 u'building_information_modeling_(bim) building_information_modeling',
 u'digital_signal_processing digital_signal_processors',
 u'genetic genetics',
 u'site_development website_development',
 #u'hr_software erp_software', ###################################企业管理人事管理软件
 u'buyer buyers',
 u'pharmaceutical pharmaceutics',
 u'musical_education music_education',
 #u'social_management soil_management',
 u'wireframe wireframes',
 u'engineer engineers',
 #u'p&l_management r&d_management'            
              
  
               
               
#u'communication communication_rh',
 #u'security it_security',   @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
 #u'hr_development development', @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
 #u'strategy it_strategy',  @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
 u'private_bank private_banking',
 #u'politics geopolitics',
 u'outsourcing it_outsourcing',
 #u'television ip_television',
 #u'governance it_governance',
 #u'consulting consulting_rh',
 u'engineering engineer',
 u'management management_it',
 #u'bioinformatics informatics',
 #u'sourcing outsourcing',
 #u'chemistry biochemistry',
 u'recruitment it_recruitment',
 #u'strategy hr_strategy',
 u'retail_bank retail_banking',
 u'architecture architecture_si',
 u'management it_management',
 #u'hr_policies policies',
 u'derivatives fx_derivatives',
 u'modeling 3d_modeling',
 u'architecture it_architecture',
 #u'marketing webmarketing',
 #u'management hr_management',
 u'insurance reinsurance',
 u'team_work -_team_work',
 u'placement outplacement',
 u'consulting hr_consulting']

In [84]:
################################################
#abnormal spell,wrong spell or wrong translation
#################################################
for x in key_save.keys():
    if "\\" in repr(x):
        #print repr(x)
        #print translate(x.encode('unicode-escape').decode('string_escape'),'en','fr')
        if fre_save[x]>1:
            print fre_save[x]
            print repr(x)

In [88]:
to_delete=[]
for x in key_save.keys():
    if "\\" in repr(x):
        print repr(x)
        to_delete.append(x)

for x in tqdm(to_delete):
    del key_save[x]
    del fre_save[x]

In [5]:
###########################################################
#####10 most similar skills of each skill in key_save.keys()
############################################################
key2similar = {}
for x in tqdm(key_save.keys()):
    if x in word2vec:
        key2similar[x] = [m[0] for m in word2vec.most_similar([x])]

A Jupyter Widget

D:\anaconda2\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until
D:\anaconda2\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  after removing the cwd from sys.path.
D:\anaconda2\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [35]:
####################################################################
#temporarily consider the top skills and the other
####################################################################
#@njit
#def Df_maker(key_save,key2similar,data_profile):
key_list = key_save.keys()
df = pd.DataFrame(data=[],index=[],columns=key_list)
start = time.clock()
initia_rang = np.zeros(len(key_list))

#for i in tqdm(range(len(data_profile))):
for k in tqdm(range(0,80)):
    for i in tqdm(range(500*k,500*k+500)):   
        rang = np.zeros(len(key_list))     ########init_rang?????????????????????????????????????????????????????????????????????????????????????????????????????????????
        if 'skills' in data_profile[i].keys():
            for skills_divided in data_profile[i]['skills']:
                if skills_divided['title'] == 'Top Skills':
                    for j in ((range(len(skills_divided['skills'])))):
                        for x in key_list:
                            if skills_divided['skills'][j]['title'] in key_save[x]:
                                index = key_list.index(x)             
                                rang[index] = 1
                                if x in key2similar:
                                    mostsimi_skill = key2similar[x]
                                for  simil_skill in mostsimi_skill:
                                    if simil_skill in key_list:
                                        index = key_list.index(simil_skill) 
                                        rang[index] += 0.75
                else:
                    for j in ((range(len(skills_divided['skills'])))):
                        for x in key_list:
                            if skills_divided['skills'][j]['title'] in key_save[x]:
                                index = key_list.index(x)             
                                rang[index] += 0.4                                                           
        df.loc[i] = rang
    df.to_csv(str(k)+'.csv')
    df = pd.DataFrame(data=[],index=[],columns=key_list)
print(time.clock()-start) 
#return df,time.clock()-start
df.to_csv('df_skill.csv')

A Jupyter Widget

A Jupyter Widget

D:\anaconda2\lib\site-packages\ipykernel_launcher.py:20: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


A Jupyter Widget

A Jupyter Widget

...

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget


3136.05829898


In [3]:
df = pd.read_csv('df_skill.csv',index_col=0)

In [4]:
df.head()

,seo_copywriting,sme_management,business_finance,t_wood,business_relations,cobit,process_management,elevator_pitch,electricity,scheduling_of_production_workshops,...,english:_at_ease_in_a_professional_situation_with_technical_vocabulary,financial_mathematics,public_business_law,devops_-_continuous_integration,pumps,independent_living,debian,filter_planted_with_plants,sass,security_law
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.4,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
